<img src="images/Cover_BinanceAPI_OrderBook.png">

# The Binance API

# Order book data

## Introduction

Hello, data miners! ⛏

The second episode of this chapter regarding the Binance API will explore just one, always useful, piece of information you can retrieve: the Order Book. If you read these articles, you are probably very familiar with the concept of an Order Book so I will not bother you with all the usual boiler-plate introduction on what is an Order Book? How Orders are matched in Finance? and bla and bla and bla.

I do not have to rank on Google so I will keep my posts as I think it's better: concise. Very few concepts per article, some small examples, and a max of 5 minutes read.

By the way, my [the last article](https://gcgrossi.github.io/NoMore_Excel_has_stopped_working/Binanceapi_intro/) on the Binance API came just before the collapse of the Terra ecosystem, with the de-peg of its stable coin UST and the annihilation of its token LUNA, which value went from 120$ to zero in record time! 

Since then, a recovery plan has been rolled out, and a new version of the coin (LUNA-2) has been released and it's trading at almost 6$ by the time I'm writing. Since the near future of this token will probably be very volatile, I think a nice, and funny aspect to cover is live monitoring.

Live monitoring comes into play in any aspect of data and is of crucial importance in many fields where the live performance of any kind of system should be evaluated on the fly, to make snap decisions (I think about Formula1 telemetry here). When I was working at CERN as a physicist of the Large Hadron Collider, I designed a (small) part of the live dashboarding system, that was able to stream the live performance of the particle detectors in the experiment, while the machine was running and taking data.

In Finance almost everything is live and stock data can be obtained with the frequency of less than minutes so it makes a perfect use case for practicing with live dashboarding. This is what we're going to attack today! 

In the next episodes, we will build a simple dashboard to stream live data from cryptocurrencies. We will start by designing the very first one: a table with the order book that is constantly updated. What stack we'll be using? You may think about something very complicated but listen to this:

**WE WILL ONLY USE PYTHON** 🐍

We will leverage the power of Pandas and Jupiter Notebook widgets, so let's just create a blank Jupiter Notebook and start drawing on it like an artist with an empty canvas! FYI: this article is completely written with Jupyter Notebook!

# Retrieveing the first data

retrieving the order book for the couple LUNA/BUSD is done via the endpoint 'depth'. In the previous article, we learnt how to perform the request to get the data, so the next cell should be very familiar to you.

In [1]:
import json
import requests

r = requests.get("https://api.binance.com/api/v3/depth",params=dict(symbol="LUNABUSD"))
response = r.json()
response

{'lastUpdateId': 1514402319,
 'bids': [['6.62410000', '3.18000000'],
  ['6.62380000', '196.56000000'],
  ['6.62370000', '3.08000000'],
  ['6.62260000', '18.48000000'],
  ['6.62170000', '1.52000000'],
  ['6.62020000', '64.16000000'],
  ['6.62010000', '60.81000000'],
  ['6.62000000', '716.02000000'],
  ['6.61980000', '2.18000000'],
  ['6.61970000', '2.81000000'],
  ['6.61960000', '27.08000000'],
  ['6.61880000', '439.93000000'],
  ['6.61810000', '33.00000000'],
  ['6.61800000', '8.00000000'],
  ['6.61760000', '247.00000000'],
  ['6.61740000', '389.37000000'],
  ['6.61730000', '3.62000000'],
  ['6.61510000', '4.38000000'],
  ['6.61500000', '8.86000000'],
  ['6.61490000', '165.47000000'],
  ['6.61420000', '1.59000000'],
  ['6.61410000', '75.46000000'],
  ['6.61400000', '5.00000000'],
  ['6.61310000', '2.17000000'],
  ['6.61250000', '7.27000000'],
  ['6.61180000', '382.89000000'],
  ['6.61150000', '319.44000000'],
  ['6.61050000', '388.94000000'],
  ['6.61010000', '490.91000000'],
  ['6.610

The response is always a dictionary. In this case, the "bids/asks" key is a list of lists with the holding price, and quantity values.

Let's just construct the bid and ask DataFrames by reading the "bid"/"ask" key of the response. Using the pandas method ```from_records``` the corresponding list can be transformed into a DataFrame with a snap! 

As the name suggests, with the method ```concat``` we can concatenate the two DataFrames.

In [2]:
import pandas as pd

# define the bid/ask datframes by reading the bid/ask keys of the response
# and the pandas constructor method from_records. Add also a column "side".
df_bid = pd.DataFrame.from_records(response['bids'], columns=['price', 'quantity']).astype('float')
df_bid['side'] = 'bid'

df_ask = pd.DataFrame.from_records(response['asks'], columns=['price', 'quantity']).astype('float')
df_ask['side'] = 'ask'

# concatenate the dataframes
# for the bid we first sort the dataframe in ascending order
# then we select the last 5 rows of bid and the first five of ask
df = pd.concat([df_bid.sort_values(by=['price'])[-5:], df_ask[:5]], ignore_index=True)
df

,price,quantity,side
0,6.6217,1.52,bid
1,6.6226,18.48,bid
2,6.6237,3.08,bid
3,6.6238,196.56,bid
4,6.6241,3.18,bid
5,6.6290,21.10,ask
6,6.6296,13.47,ask
7,6.6301,15.07,ask
8,6.6307,3.02,ask
9,6.6318,3.02,ask


As you can see we limited the number of entries to 5 for each table, and we sorted the bid prices in ascending order to have the effect of bid-ask matching that you often see in the order book representations.

## Representing the table in ipython widgets

We will now start working with [ipython widgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html) library. The library has a widget that can hold HTML values, and that is really what we are looking for in this project. Why?

Not a lot of people know that pandas DataFrames have an API (meaning an entire set of functions) to style the DataFrame and export it to HTML.
There is a nice step-by-step guide on the [official website](https://pandas.pydata.org/docs/user_guide/style.html).
As a first step let's see how a DataFrame is rendered in HTML by simply creating an HTML widget and dumping the DataFrame with the ```to_html()``` method.

In [3]:
import ipywidgets as widgets

# create a jupyter widget with the table html
widgets.HTML(value=df.to_html())

HTML(value='<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></…

Well, it's rendered like ... a DataFrame. We can play a little bit more with the API and eliminate the index, alongside selected columns with the methods ```hide_index()``` and ```hide_columns()```. The methods return a 'style' object (basically an HTML table with some CSS) that can be rendered in Jupyter Notebooks or other engines.

In [4]:
# hiding the index and the column 'side'
# st is a 'style' object. 
st=df.style.hide_index()
st = st.hide_columns(['side'])
widgets.HTML(value=st.to_html())

HTML(value='<style type="text/css">\n</style>\n<table id="T_3a8af_">\n  <thead>\n    <tr>\n      <th class="co…

## Draw the table in black green and red colors

Now it's time to make some cool styling. In the following cell, we are going to add colors to the table. Since the styler object is an HTML table with CSS for styling, we need to modify the CSS of the table. For those who are not familiar with CSS, I will just tell you that is a language that can select HTML elements based on some rules, and add style attributes to them. In example, I may select all the ```<p>``` tags in an HTML file and apply the style ```color:rgb(0,0,0)```. 

But how can we do that in a Pandas DataFrame? The styler object can add CSS! Using the ```set_table_styles``` method. You will need to pass a list of CSS selectors and style properties to the method to apply the desired changes. But let's see how this is translated into code:

In [13]:
# here we create a mask for selecting bid and ask
# portions of the dataframe. We will color 
# the two parts in different ways
df_mask = df.copy(deep=True)
df_mask[df['side'] == 'bid'] = 'bid'
df_mask[df['side'] == 'ask'] = 'ask'

# here we apply the style
# pass a list of dictionaries with keys 
# 'css selector' and 'properties'
s = df.style.set_table_styles([
    {'selector': 'th:not(.index_name)','props':'text-align:center; color:grey;background-color:#373737'},  # select all the headers 
    {'selector': 'td','props':'text-align:center; background-color:#2d2d2d;font-weight:bold'},  # select all table cells
    {'selector':'.bid','props':'color:#086623'}, # select all elements with class name 'bid'
    {'selector':'.ask','props':'color:red'}, # select all elements with class 'ask'
],overwrite=False)

# we set the classes of the cells to
# 'bid' or 'ask' using the mask we created
s.set_td_classes(df_mask)

,price,quantity,side
0,29945.010000,0.166740,bid
1,29945.300000,0.015490,bid
2,29946.610000,0.000400,bid
3,29946.620000,0.030000,bid
4,29947.540000,0.026630,bid
5,29950.820000,0.675350,ask
6,29952.600000,0.050000,ask
7,29953.180000,0.050000,ask
8,29953.940000,0.166230,ask
9,29954.610000,0.039820,ask


In the previous cell, we created a mask DataFrame with strings 'bid' or 'ask' depending on if the cell should be colored with green (bid) or red (ask). We applied the desired style to the DataFrame. We first selected the headers (```th:not(.index_name)```) and applied some light grey and text centering. After we select all the cells (```td```) and apply a dark grey background. After we select all the cells with class names 'bid' and 'ask' (```.bid, .ask```) and color the font in green and red.

In the end, the ```set_td_classes``` sets the class of cells in the styler equal to the strings in the mask we previously created (we need it to select when in the CSS selector).

The result is pretty stylish! 😎

We can push the limit and add a final touch, by adding bars in the volume columns. Let's do that!

## We add the bars

We now add bars in the 'quantity' column, using the ```.bar``` method of the styler object. This will draw a bar in each cell, with height proportional to the value of the cell. We need to pass to the method the portion of the DataFrame we want to add bars. 

To do that we will use the pandas ```IndexSlice```, the object that is responsible for the .loc magic in pandas. Let's jump to the next cell to see the code:

In [14]:
# we initialize a pandas IndexSlice
idx = pd.IndexSlice

# we retrieve the indexes of the 'bid' portion of the
# DataFrame as a list
idxs = df.index[df['side'] == 'bid'].tolist()

# Here we use the .bar method. We provide the method
# with the subset of the Datframe to apply bars in the same
# way we would do in .loc, but using the IndexSlice.
# we also provide the color of the bar as 2nd argument
s.bar(subset=idx[:idxs[-1],'quantity'], color='#dbeac5')
s.bar(subset=idx[idxs[-1]+1:,'quantity'], color='#ff9090')

,price,quantity,side
0,29945.010000,0.166740,bid
1,29945.300000,0.015490,bid
2,29946.610000,0.000400,bid
3,29946.620000,0.030000,bid
4,29947.540000,0.026630,bid
5,29950.820000,0.675350,ask
6,29952.600000,0.050000,ask
7,29953.180000,0.050000,ask
8,29953.940000,0.166230,ask
9,29954.610000,0.039820,ask


Easy peasy. In a few lines of code, we Initialize a pandas IndexSlice and we use it in the ```.bar``` method to select the portion of the DataFrame that should have bars. We also pass a custom color. The result is nice I think!

We will use the HTML of this table in the next step to fill an ipython HTML widget, but the job is pretty much done!

## Drawing the Jupyter Widget

Before drawing the widget, I add some CSS with Jupyter magic, just to have a uniform background for the widget. 

I then create a horizontal box widget with the style I created and some additional Layout and set his children to ```tab```, the HTML widget that is holding the table. 

In [17]:
%%html
<style>
    .box_style{
        width:100%;
        border : 0px solid red;
        height: auto;
        background-color:#373737;
    }
</style>

In [19]:
from ipywidgets import Layout

# this is the widget holding the HTML of the table
tab = widgets.HTML(value=s.to_html())

# this is an horizontal box holding the table widget
# with some Layout an the CSS style we defined before
hBox = widgets.HBox([tab],layout=Layout(justify_content= 'center'))
hBox.add_class("box_style")

# draw the horizontal Box
hBox

Et voilà! Let's know put some magic on this table and animate it. 🧙‍♂️

## 🧙‍♂️ Let the magic begin: updating the table with live data

Nothing more than wrapping up all the previous cells in a function called ```style_pipe```. The function takes a DataFrame in input and returns the styler object with all the properties we built before.

In [20]:
def style_pipe(df):
    # this is just a copy paste of the previous cells
    # the functions returns a styler objet 

    # hide index and columns
    s= df.style.hide_index().hide_columns(['side'])

    # color the cells in red, gree and apply style
    # to index and headers
    df_mask = df.copy(deep=True)
    df_mask[df['side'] == 'bid'] = 'bid'
    df_mask[df['side'] == 'ask'] = 'ask'

    s.set_table_styles([
        {'selector': 'th:not(.index_name)','props':'text-align:center; color:grey;background-color:#373737'},
        {'selector': 'td','props':'text-align:center; background-color:#2d2d2d;font-weight:bold'},
        {'selector':'.bid','props':'color:green'},
        {'selector':'.ask','props':'color:red'},
    ],overwrite=False)

    s.set_td_classes(df_mask)   

    # add the bars in the bid and ask portion
    # of the dataframe
    idxs = df.index[df['side'] == 'bid'].tolist()
    idx = pd.IndexSlice
    s.bar(subset=idx[:idxs[-1],'quantity'], color='#dbeac5')
    s.bar(subset=idx[idxs[-1]+1:,'quantity'], color='#ff9090')

    return s 

Then we launch an 'infinite' loop that retrieves the data from Binance, constructs the Order Book DataFrame, and applies the style pipeline to the HTML in the ```tab``` widget we just defined. Pretty much all we did in this notebook is wrapped up!

In [21]:
# here we just repeat the process of retrieving data
# applying the style
# and overwrite the html in the tab widget

# start a long loop (not using while True 
# for obvious reasons)
count=0
while count<100:
    
    # retrieve data from Binance
    r = requests.get("https://api.binance.com/api/v3/depth",params=dict(symbol="LUNABUSD"))
    response = r.json()

    # construct the order book Dataframe
    df_bid = pd.DataFrame.from_records(response['bids'], columns=['price', 'quantity']).astype('float')
    df_bid['side'] = 'bid'

    df_ask = pd.DataFrame.from_records(response['asks'], columns=['price', 'quantity']).astype('float')
    df_ask['side'] = 'ask'

    df = pd.concat([df_bid.sort_values(by=['price'])[-5:], df_ask[:5]], ignore_index=True)
    count=count+1
    
    # updating the widget HTML after
    # having applied the style the DataFrame
    tab.value=style_pipe(df).to_html()

## Conclusion

If everything went fine, while the last cell was running, your widget got updated in real-time, by the time the loop was still processing. I think this makes a lot of fun because from here we can construct a complete and even complex dashboard! And get this:

we did everything in a Jupyter Notebook, with just some basic Python and the help of HTML and CSS. I don't know you but I am sincerely impressed by how far technology has pushed itself! 

Pure Python magic! 🧙‍♂️

I think next time I will add some plots to this widget! I want to see how far I can go. This always happens to me when playing with data and tech: I take a road, and I cannot see the end ... 

I want to see where it goes ... 

I want to see if I can make it to the next hill or turn.

and there I go for my adventure! 

I may come back? I don't know. 

I will be the same when back home? Never!

As Bilbo sang one time:

*The Road goes ever on and on,* <br>
*Down from the door where it began.* <br>
*Now far ahead the Road has gone,* <br>
*And I must follow, if I can,* <br>
*Pursuing it with eager feet,* <br>
*Until it joins some larger way* <br>
*Where many paths and errands meet.* <br>
*And whither then? I cannot say.*